# Tutorial

This tutorial demonstrates the main workflow for using `mpllayout`.

The workflow involves just a few high-level steps:
1. Create a layout object to store the layout, `layout = Layout()`
2. Add geometric primitives to `layout` using `layout.add_prim`. These primitives represent figure elements.
3. Add geometric constraints to `layout` using `layout.add_constraint` to constrain the primitives.
4. Solve the constrained layout of primitives using `constrained_prims = solve(layout.root_prim, *layout.flat_constraints())`
5. Generate a figure and axes to plot in using `fig, axs = subplots(constrained_prims)`

The generated `fig` and `axs` will reflect the constrained layout.

In [ ]:
import numpy as np

import matplotlib as mpl
from matplotlib import pyplot as plt

# `layout` contains the `Layout` class and related functions
from mpllayout import layout as lay
# `primitives` contains primitives and `constraints` constraints
from mpllayout import primitives as pr
from mpllayout import constraints as co
# `solve` is used to solve the constrained layout
from mpllayout.solver import solve

# `subplots` and `update_subplots` are used to create matplotlib figure and
# axes objects from geometric primitives
from mpllayout.matplotlibutils import subplots, update_subplots

# `ui` contains functions to visualize primitives
from mpllayout import ui

## Step 1: Create the layout

In [ ]:
# Create the layout to store constraints and primitives
layout = lay.Layout()

## Step 2: Add geometric primitives

Geometric primitives represent geometry and are defined in `mpllayout.primitives`.
Each primitive consists of a parameter vector (`primitive.value`) with child primitives (`primitive["child_key"]`).
For example:

* `Point` represents a point and has a parameter vector containing its coordinates with no child primitives
* `Line` represents a straight line segment, has no parameter vector, and contains two points representing the start point (`line["Point0"]`) and end point (`line["Point1"]`)
* Other primitives are documented in the module

Geometric primitives are added using the call
`layout.add_prim(primitive, key)`
where `primitive` is a geometric primitive object and `key` is a string used to identify it.

In [ ]:
# A `Quadrilateral` is a 4 sided polygon which can be used to represent the figure box.
# Naming the quad "Figure" will cause the `subplots` command to create a figure of the same size.
layout.add_prim(pr.Quadrilateral(), "Figure")

# The `Axes` primitive is a collection of quadrilaterals and points used to represent an axes.
# The child primitives of `Axes` are
# - "Frame": a `Quadrilateral` representing the plotting area of the axes
# - "XAxis": a `Quadrilateral` bounding x-axis ticks and tick labels
# - "XAxisLabel": a `Point` for the x-axis label text anchor
# - "YAxis": a `Quadrilateral` bounding y-axis ticks and tick labels
# - "YAxisLabel": a `Point` for the y-axis label text anchor
# The x/y axis can be optionally included by kwargs as seen below
layout.add_prim(pr.Axes(xaxis=True, yaxis=True), "Axes")

## Step 3: Add geometric constraints

Geometric constraints represent constraints or conditions on primitives and are defined in `mpllayout.constraints`.
Every constraint has a method representing the condition
`Constraint.assem_res(prims, **kwargs)`, 
where `prims` is a tuple of primitives the constraint applies to and `**kwargs` are constraint specific parameters.
The constraint is satisfied when `assem_res` is 0.

For example:

* `Coincident().assem_res((pointa, pointb))` represents the coincidence error between two `pointa` and `pointb` (no parameters are needed).
* `Length().assem_res((linea,), length=7)` represents the length error for `linea` compared to the desired length of 7.
* Other constraints are documented in the module

Geometric constraints are added to a layout using the call
`layout.add_constraint(constraint, prim_keys, constraint_params)`,
where 

* `constraint` is the geometric constraint object
* `prim_keys` is a tuple of primitive keys representing the primitives to constrain
* `constraint_params` is a dictionary or tuple of constraint specific parameters.

`prim_keys` can recursively indicate primitives uses slash separated keys.
For example the tuple `("Figure/Line0/Point0", "Axes/Frame/Line0")` represents the point 0 of the figure quadrilateral (the bottom left) and line 0 of the axes frame quadrilateral (the bottom line).

`constraint_params` represents the `**kwargs` of `assem_res`.
This can be either a dictionary or tuple representing the kwargs.

The next few sections add sets of constraints and plot the resulting constrained layout to illustrate their effect.


### Make all quadrilaterals rectangular

`Quadrilateral`s have 4 unknown coordinates for each corner. 
To make them rectangular boxes like axes and figures, apply the `Box` constraint.

In [ ]:
# NOTE: This step is needed because `Quadrilateral` by default don't have
# to be rectangles
layout.add_constraint(co.Box(), ("Figure",), ())
layout.add_constraint(co.Box(), ("Axes/Frame",), ())
layout.add_constraint(co.Box(), ("Axes/XAxis",), ())
layout.add_constraint(co.Box(), ("Axes/YAxis",), ())

In [ ]:
# The layout currently looks like:
_fig, _ = ui.figure_prims(solve(layout)[0], fig_size=(5, 5))

### Fix the figure position and size

In [ ]:
# Fix the figure bottom left to the origin
layout.add_constraint(co.Fix(), ("Figure/Line0/Point0",), (np.array([0, 0]),))

# Figure the figure width and height
fig_width, fig_height = 6, 3
layout.add_constraint(co.XLength(), ("Figure/Line0",), (fig_width,))
layout.add_constraint(co.YLength(), ("Figure/Line1",), (fig_height,))

In [ ]:
# The layout currently looks like:
_fig, _ = ui.figure_prims(solve(layout)[0], fig_size=(5, 5))

### Position the x and y axis and axis labels

In [ ]:
## Position the x axis on top and the y axis on the bottom
# When creating axes from the primitives, `subplots` will detect axis
# positions and set axis properties to reflect them.
layout.add_constraint(co.PositionXAxis(side='top'), ("Axes", ), ())
layout.add_constraint(co.PositionYAxis(side='right'), ("Axes", ), ())

# Link x/y axis width/height to axis sizes in matplotlib.
# Axis sizes change depending on the size of their tick labels so the
# axis width/height must be linked to matplotlib and updated from plot
# elements.
layout.add_constraint(
    co.XAxisHeight(), ("Axes/XAxis",), (None,),
)
layout.add_constraint(
    co.YAxisWidth(), ("Axes/YAxis",), (None,),
)

## Position the x/y axis label text anchors
# When creating axes from the primitives, `subplots` will detect these and set
# their locations
on_line = co.RelativePointOnLineDistance()
to_line = co.PointToLineDistance()

## Pad the x/y axis label from the axis bbox
pad = 1/16
layout.add_constraint(to_line, ("Axes/XAxisLabel", "Axes/XAxis/Line2"), (True, pad))
layout.add_constraint(to_line, ("Axes/YAxisLabel", "Axes/YAxis/Line1"), (True, pad))

## Center the axis labels halfway along the axes width/height
layout.add_constraint(co.PositionXAxisLabel(), ("Axes",), (0.5,))
layout.add_constraint(co.PositionYAxisLabel(), ("Axes",), (0.5,))

In [ ]:
# The layout currently looks like:
_fig, _ = ui.figure_prims(solve(layout)[0], fig_size=(5, 5))

### Set margins between the axes and figure

In [ ]:
## Constrain margins around the axes to the figure
# Constrain left/right margins
margin_left = 0.1
margin_right = 1/4

layout.add_constraint(
    co.InnerMargin(side='left'), ("Axes/Frame", "Figure"), (margin_left,)
)
layout.add_constraint(
    co.InnerMargin(side='right'), ("Axes/YAxis", "Figure"), (margin_right,)
)

# Constrain top/bottom margins
margin_top = 1/4
margin_bottom = 0.1
layout.add_constraint(
    co.InnerMargin(side='bottom'), ("Axes/Frame", "Figure"), (margin_bottom,)
)
layout.add_constraint(
    co.InnerMargin(side='top'), ("Axes/XAxis", "Figure"), (margin_top,)
)


In [ ]:
# The layout currently looks like:
_fig, _ = ui.figure_prims(solve(layout)[0], fig_size=(5, 5))

## Step 4: Solve the constrained layout 

In [ ]:
## Solve the constraints and form the figure/axes layout
prim_tree_n, solve_info = solve(layout)

print(f"Absolute errors: {solve_info['abs_errs']}")
print(f"Relative errors: {solve_info['rel_errs']}")

## Step 5: Plot into the figure and axes 

In [ ]:
## Plot into the generated figure and axes
fig, axs = subplots(prim_tree_n)

x = np.linspace(0, 1)
axs["Axes"].plot(x, x**2)

axs["Axes"].xaxis.set_label_text("My x label", ha="center", va="bottom")
axs["Axes"].yaxis.set_label_text("My y label", ha="center", va="bottom", rotation=-90)

ax = axs["Axes"]

# Using the generated axes and x/y axis contents, the layout constraints
# can be updated with those matplotlib elements
layout = lay.update_layout_constraints(layout, axs)
prim_tree_n, solve_info = solve(layout)

# This updates the figure and axes using the updated layout
update_subplots(prim_tree_n, "Figure", fig, axs)

In [ ]:
# The layout currently looks like:
_fig, _ = ui.figure_prims(solve(layout)[0], fig_size=(5, 5))

# Note that x and y axis dimensions have adjusted